<a href="https://colab.research.google.com/github/abdullahmujahidali/Vet-LangGraph/blob/main/VetAI_Langchain_With_Indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q langchain langgraph langchain_openai openai python-dotenv pinecone langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [10]:
import os
from typing import List, Dict, Tuple, Annotated, TypedDict, Union, Any, Optional
from datetime import datetime
from enum import Enum
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone
from google.colab import userdata
import json


OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)


index_name = "veterinary-ai-index"
embeddings = OpenAIEmbeddings()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

def query_pinecone(query: str, top_k: int = 5) -> List[dict]:
    """Query Pinecone index for relevant context"""
    query_embedding = embeddings.embed_query(query)
    index = pc.Index(index_name)

    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )

    return results.matches


class DiagnosisType(str, Enum):
    DEFINITIVE = "definitive"
    DIFFERENTIAL = "differential"
    SYMPTOM_BASED = "symptom_based"
    LAB_ABNORMALITY = "lab_abnormality"

class ConfidenceLevel(str, Enum):
    CONFIRMED = "confirmed"
    PRESUMPTIVE = "presumptive"
    OWNER_REPORTED = "owner_reported"

class ProblemStatus(str, Enum):
    ACTIVE = "active"
    HISTORICAL = "historical"
    RESOLVED = "resolved"
    RELAPSED = "relapsed"

class TreatmentResponse(str, Enum):
    IMPROVED = "improved"
    WORSENED = "worsened"
    NO_CHANGE = "no_change"
    SIDE_EFFECTS = "side_effects"

class Medication(TypedDict):
    drug_name: str
    formulation: str
    concentration: str
    dose_given: str
    frequency: str
    duration: str
    calculated_dose_mg_per_kg: Optional[float]
    response: TreatmentResponse
    owner_perception: str
    side_effects: str

class Problem(TypedDict):
    description: str
    type: DiagnosisType
    confidence: ConfidenceLevel
    status: ProblemStatus
    onset_date: str
    progression: str
    related_symptoms: List[str]
    treatments: List[Medication]

class LabResult(TypedDict):
    test_name: str
    value: str
    reference_range: str
    unit: str
    flag: str
    date: str

class VetState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    current_input: str

    medical_records: Dict[str, Any]
    owner_history: Dict[str, Any]
    diagnoses: Dict[str, Problem]
    lab_results: Dict[str, LabResult]

    medications: Dict[str, Medication]
    treatment_history: Dict[str, Any]

    results: Dict[str, Any]

    metadata: Dict[str, Any]
    security_flags: Dict[str, Any]

    calculations: Dict[str, Any]
    trends: Dict[str, Any]

    owner_constraints: Dict[str, Any]
    owner_preferences: Dict[str, Any]
    context: List[str]





In [23]:
def process_data(state: VetState) -> VetState:
    relevant_docs = query_pinecone(state["current_input"])
    context = [doc.metadata.get("text", "") for doc in relevant_docs]
    context_text = "\n\n".join(context)

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary data processing agent responsible for initial data extraction and organization.
        Use the following context from previous medical records to inform your analysis: {context}

KEY RESPONSIBILITIES:
1. File Processing:
   - Process veterinary records, lab reports, and supporting documents
   - Extract and structure content based on document type
   - Apply standardized formatting
   - Generate metadata tags for indexing

2. Content Organization:
   - Patient Information (Name, Species, Breed, Age, ID)
   - Clinical History & Physical Exam findings
   - SOAP Notes and Progress Reports
   - Medications & Treatments (with precise dosing)
   - Laboratory Results (with reference ranges)
   - Diagnostic Imaging Reports
   - Doctor's Recommendations

3. Data Security:
   - Identify and flag sensitive information
   - Apply appropriate redaction where needed
   - Note areas requiring manual review

OUTPUT FORMAT:
Generate a structured output containing:
1. Parsed medical data in standardized format
2. Metadata tags for indexing
3. Security/privacy flags
4. Data quality indicators"""),
        ("human", "Process and structure the following veterinary input: {input}")
    ])

    try:
        messages = prompt.format_messages(
            input=state["current_input"],
            context=context_text)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["DataProcessor"] = response.content
        new_state["metadata"]["processing_timestamp"] = datetime.now().isoformat()
        new_state["context"] = context

        return new_state

    except Exception as e:
        return handle_error(state, f"Data Processing Error: {str(e)}")

def analyze_history(state: VetState) -> VetState:
    context = state.get("context", [])
    context_text = "\n\n".join(context)
    history_context = query_pinecone(state["results"]["DataProcessor"])
    additional_context = [doc.metadata.get("text", "") for doc in history_context]
    combined_context = context + additional_context
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary history analysis agent specializing in comprehensive medical history evaluation.
        Consider this additional context from similar cases: {context}

KEY RESPONSIBILITIES:
1. Medical History Analysis:
   - Extract all past and current medical conditions
   - Categorize by body system
   - Document onset dates, severity, progression
   - Calculate precise medication dosing

2. Problem Categorization:
   - Active vs Historical Problems
   - Related vs Separate Issues
   - Primary vs Secondary Conditions
   - Track progression status

3. Treatment History:
   - Document all medications with exact details
   - Calculate mg/kg dosing where applicable
   - Track treatment responses
   - Note adverse reactions

4. Owner Information:
   - Document reported symptoms
   - Note owner concerns
   - Track compliance history
   - Record practical limitations"""),
        ("human", "Based on this processed data and historical context, provide a comprehensive medical history analysis: {processed_data}")
    ])

    try:
        processed_data = state["results"]["DataProcessor"]
        messages = prompt.format_messages(
            processed_data=processed_data,
            context="\n\n".join(combined_context)
        )
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["HistoryAnalyzer"] = response.content
        new_state["context"] = combined_context

        return new_state

    except Exception as e:
        return handle_error(state, f"History Analysis Error: {str(e)}")

def analyze_clinical(state: VetState) -> VetState:
    context = state.get("context", [])
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary clinical record analyst responsible for comprehensive medical record review and synthesis.
         Consider this historical context when analyzing: {context}

KEY RESPONSIBILITIES:
1. Record Analysis:
   - Review all medical documentation
   - Track patient details and trends
   - Analyze laboratory results
   - Document owner compliance

2. Data Trending:
   - Track weight and BCS changes
   - Monitor lab value trends
   - Document treatment responses
   - Note disease progression

3. Pattern Recognition:
   - Identify related symptoms
   - Track disease progression
   - Note treatment responses
   - Flag unusual presentations"""),
        ("human", "Based on this history analysis, provide a detailed clinical analysis: {history_analysis}")
    ])

    try:
        history_analysis = state["results"]["HistoryAnalyzer"]
        messages = prompt.format_messages(
            history_analysis=history_analysis,
            context="\n\n".join(context)
        )
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["ClinicalAnalyzer"] = response.content

        return new_state

    except Exception as e:
        return handle_error(state, f"Clinical Analysis Error: {str(e)}")

def diagnose(state: VetState) -> VetState:
    context = state.get("context", [])
    clinical_analysis = state["results"]["ClinicalAnalyzer"]
    diagnostic_context = query_pinecone(clinical_analysis)
    additional_context = [doc.metadata.get("text", "") for doc in diagnostic_context]
    combined_context = context + additional_context

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary diagnostic intelligence agent responsible for analyzing and categorizing medical problems with confidence levels.
         Consider this clinical context from similar cases: {context}


KEY RESPONSIBILITIES:
1. Problem Classification:
   - Categorize diagnoses by type
   - Assign confidence levels
   - Document progression patterns
   - Identify related issues

2. Diagnostic Analysis:
   - Cross-reference symptoms
   - Evaluate test results
   - Consider breed-specific issues
   - Factor in age-related concerns"""),
        ("human", "Based on this clinical analysis, provide diagnostic insights and recommendations: {clinical_analysis}")
    ])

    try:
        messages = prompt.format_messages(
            clinical_analysis=clinical_analysis,
            context="\n\n".join(combined_context)
        )
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["Diagnostics"] = response.content
        new_state["context"] = combined_context

        return new_state

    except Exception as e:
        return handle_error(state, f"Diagnostic Error: {str(e)}")


def handle_error(state: VetState, error_message: str) -> VetState:
    new_state = state.copy()
    if "errors" not in new_state["results"]:
        new_state["results"]["errors"] = []

    new_state["results"]["errors"].append({
        "timestamp": datetime.now().isoformat(),
        "message": error_message,
        "context": state.get("context", [])  # Include context in error logging
    })

    return new_state

def calculate_medication_dosing(medications: List[Dict], weight: Optional[float]) -> Dict:
    if not weight:
        return {"error": "No weight available for dosing calculations"}

    results = {}
    for med in medications:
        try:
            dose_str = med.get("dose_given", "")
            dose_num = float(''.join(filter(str.isdigit, dose_str)))

            mg_per_kg = dose_num / weight

            results[med["drug_name"]] = {
                "mg_per_kg": round(mg_per_kg, 2),
                "calculated_from": {
                    "dose": dose_num,
                    "weight": weight
                }
            }
        except Exception as e:
            results[med["drug_name"]] = {
                "error": f"Calculation failed: {str(e)}"
            }

    return results

def create_data_processing_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary data processing agent responsible for initial data extraction and organization.
         Use the following historical context to inform your assessment: {context}

KEY RESPONSIBILITIES:
1. File Processing:
   - Process veterinary records, lab reports, and supporting documents
   - Extract and structure content based on document type
   - Apply standardized formatting
   - Generate metadata tags for indexing

2. Content Organization:
   - Patient Information (Name, Species, Breed, Age, ID)
   - Clinical History & Physical Exam findings
   - SOAP Notes and Progress Reports
   - Medications & Treatments (with precise dosing)
   - Laboratory Results (with reference ranges)
   - Diagnostic Imaging Reports
   - Doctor's Recommendations

3. Data Security:
   - Identify and flag sensitive information
   - Apply appropriate redaction where needed
   - Note areas requiring manual review"""),
        ("human", "Process and structure the following veterinary input: {input}")
    ])

    def process_data(state: VetState):
        try:
            relevant_docs = query_pinecone(state["current_input"])
            context = [doc.metadata.get("text", "") for doc in relevant_docs]
            context_text = "\n\n".join(context)

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary data processing agent responsible for initial data extraction and organization.
                Use the following context from previous medical records to inform your analysis: {context}

                KEY RESPONSIBILITIES:
                1. File Processing:
                   - Process veterinary records, lab reports, and supporting documents
                   - Extract and structure content based on document type
                   - Apply standardized formatting
                   - Generate metadata tags for indexing

                2. Content Organization:
                   - Patient Information (Name, Species, Breed, Age, ID)
                   - Clinical History & Physical Exam findings
                   - SOAP Notes and Progress Reports
                   - Medications & Treatments (with precise dosing)
                   - Laboratory Results (with reference ranges)
                   - Diagnostic Imaging Reports
                   - Doctor's Recommendations

                3. Data Security:
                   - Identify and flag sensitive information
                   - Apply appropriate redaction where needed
                   - Note areas requiring manual review"""),
                ("human", "Process and structure the following veterinary input: {input}")
            ])

            messages = prompt.format_messages(
                input=state["current_input"],
                context=context_text
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["DataProcessor"] = response.content
            new_state["metadata"]["processing_timestamp"] = datetime.now().isoformat()
            new_state["context"] = context

            return new_state

        except Exception as e:
            return handle_error(state, f"Data Processing Error: {str(e)}")

    return process_data

def create_history_analysis_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary history analysis agent specializing in comprehensive medical history evaluation.
         Consider this historical context when analyzing connections: {context}

KEY RESPONSIBILITIES:
1. Medical History Analysis:
   - Extract all past and current medical conditions
   - Categorize by body system
   - Document onset dates, severity, progression
   - Calculate precise medication dosing

2. Problem Categorization:
   - Active vs Historical Problems
   - Related vs Separate Issues
   - Primary vs Secondary Conditions
   - Track progression status"""),
        ("human", "Based on this processed data, provide a comprehensive medical history analysis: {processed_data}")
    ])

    def analyze_history(state: VetState):
        try:
            processed_data = state["results"]["DataProcessor"]
            messages = prompt.format_messages(processed_data=processed_data)
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["HistoryAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"History Analysis Error: {str(e)}")

    return analyze_history

def create_clinical_analysis_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary clinical record analyst responsible for comprehensive medical record review and synthesis.

KEY RESPONSIBILITIES:
1. Record Analysis:
   - Review all medical documentation
   - Track patient details and trends
   - Analyze laboratory results
   - Document owner compliance

2. Data Trending:
   - Track weight and BCS changes
   - Monitor lab value trends
   - Document treatment responses
   - Note disease progression"""),
        ("human", "Based on this history analysis, provide a detailed clinical analysis: {history_analysis}")
    ])

    def analyze_clinical(state: VetState):
        try:
            history_analysis = state["results"]["HistoryAnalyzer"]
            messages = prompt.format_messages(history_analysis=history_analysis)
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["ClinicalAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Clinical Analysis Error: {str(e)}")

    return analyze_clinical

def create_diagnostic_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary diagnostic intelligence agent responsible for analyzing and categorizing medical problems with confidence levels.

KEY RESPONSIBILITIES:
1. Problem Classification:
   - Categorize by type (Definitive, Differential, Symptom-Based)
   - Assign confidence levels
   - Document progression patterns
   - Identify related issues

2. Diagnostic Analysis:
   - Cross-reference symptoms
   - Evaluate test results
   - Consider breed-specific issues
   - Factor in age-related concerns

3. Treatment Impact Assessment:
   - Evaluate therapeutic responses
   - Document treatment failures
   - Track adverse reactions
   - Monitor progression"""),
        ("human", "Based on this clinical analysis, provide diagnostic insights and recommendations: {clinical_analysis}")
    ])

    def diagnose(state: VetState):
        try:
            clinical_analysis = state["results"]["ClinicalAnalyzer"]
            context = state.get("context", [])

            # Get additional diagnostic context
            diagnostic_context = query_pinecone(clinical_analysis)
            additional_context = [doc.metadata.get("text", "") for doc in diagnostic_context]
            combined_context = context + additional_context

            messages = prompt.format_messages(
                clinical_analysis=clinical_analysis,
                context="\n\n".join(combined_context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["Diagnostics"] = response.content
            new_state["context"] = combined_context
            return new_state

        except Exception as e:
            return handle_error(state, f"Diagnostic Error: {str(e)}")

    return diagnose

def create_specialist_summary_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary specialist assessment and owner communication agent.

KEY RESPONSIBILITIES:
1. Specialist Assessment:
   - Provide detailed medical analysis of each problem
   - Define pathophysiology and likely causes
   - Assess severity and progression
   - Evaluate current treatment efficacy
   - Outline potential complications

2. Owner Communication:
   - Create clear, layman-friendly explanations
   - Explain what's wrong in simple terms
   - Outline treatment options and expectations
   - Discuss risks of non-treatment
   - Provide clear next steps

3. Treatment Planning:
   - Recommend first-line therapies
   - Suggest adjunctive treatments
   - Consider owner constraints
   - Provide monitoring criteria

OUTPUT FORMAT:
For each problem provide:
1. Specialist-Level Assessment
2. Owner-Friendly Summary
3. Treatment Plan
4. Monitoring Strategy"""),
        ("human", "Based on this diagnostic assessment, provide specialist interpretation and owner-friendly summary: {diagnostic_assessment}")
    ])

    def generate_summaries(state: VetState):
        try:
            context = state.get("context", [])
            diagnostic_assessment = state["results"].get("Diagnostics", "")

            # Get additional specialist context
            specialist_context = query_pinecone(diagnostic_assessment)
            additional_context = [doc.metadata.get("text", "") for doc in specialist_context]
            combined_context = context + additional_context

            messages = prompt.format_messages(
                diagnostic_assessment=diagnostic_assessment,
                context="\n\n".join(combined_context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["SpecialistSummary"] = response.content
            new_state["context"] = combined_context
            return new_state

        except Exception as e:
            return handle_error(state, f"Specialist Summary Error: {str(e)}")

    return generate_summaries

def create_connections_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary connections and trends analysis agent.

KEY RESPONSIBILITIES:
1. Problem Connection Analysis:
   - Identify primary vs secondary problems
   - Group related symptoms under confirmed diagnoses
   - Flag unexplained symptoms
   - Detect pattern progression

2. Case Theory Development:
   - Construct unified case theories
   - Identify underlying causes
   - Note gaps requiring investigation
   - Suggest focused workups

3. Treatment Integration:
   - Evaluate overall treatment strategy
   - Identify potential interactions
   - Suggest treatment prioritization
   - Note treatment conflicts

OUTPUT FORMAT:
1. Primary/Secondary Problem Classification
2. Unexplained Symptoms List
3. Unified Case Theory
4. Recommended Next Steps"""),
        ("human", "Based on all previous analyses, identify connections and develop a case theory: {previous_analyses}")
    ])

    def analyze_connections(state: VetState):
        context = state.get("context", [])

        # Gather all previous analyses
        previous_analyses = {
            "data_processing": state["results"]["DataProcessor"],
            "history": state["results"]["HistoryAnalyzer"],
            "clinical": state["results"]["ClinicalAnalyzer"],
            "diagnostics": state["results"]["Diagnostics"],
            "specialist": state["results"]["SpecialistSummary"]
        }

        # Get additional connection-specific context
        connection_context = query_pinecone(json.dumps(previous_analyses))
        additional_context = [doc.metadata.get("text", "") for doc in connection_context]
        combined_context = context + additional_context

        messages = prompt.format_messages(
            previous_analyses=json.dumps(previous_analyses),
            context="\n\n".join(combined_context)
        )
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["Connections"] = response.content
        new_state["context"] = combined_context
        return new_state

    return analyze_connections



In [24]:
import os
from typing import List
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def process_pdf(file_path: str) -> List[str]:
    """Process PDF document and split into chunks"""
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )

    chunks = text_splitter.split_documents(pages)
    return [chunk.page_content for chunk in chunks]

def create_workflow():
    """Create and configure the workflow"""
    workflow = StateGraph(VetState)

    # Create agents
    data_processor = create_data_processing_agent()
    history_analyzer = create_history_analysis_agent()
    clinical_analyzer = create_clinical_analysis_agent()
    diagnostic_agent = create_diagnostic_agent()
    specialist_agent = create_specialist_summary_agent()
    connections_agent = create_connections_agent()

    # Add all nodes
    if data_processor is None:
        raise ValueError("Data processor agent creation failed")
    workflow.add_node("process_data", data_processor)
    workflow.add_node("analyze_history", history_analyzer)
    workflow.add_node("analyze_clinical", clinical_analyzer)
    workflow.add_node("diagnose", diagnostic_agent)
    workflow.add_node("specialist_summary", specialist_agent)
    workflow.add_node("analyze_connections", connections_agent)

    # Set entry point
    workflow.set_entry_point("process_data")

    # Add edges
    workflow.add_edge("process_data", "analyze_history")
    workflow.add_edge("analyze_history", "analyze_clinical")
    workflow.add_edge("analyze_clinical", "diagnose")
    workflow.add_edge("diagnose", "specialist_summary")
    workflow.add_edge("specialist_summary", "analyze_connections")
    workflow.add_edge("analyze_connections", END)

    return workflow.compile()

def analyze_veterinary_case(file_path: str):
    """Main function to analyze a veterinary case"""
    try:
        # Process the PDF
        print("Processing document...")
        document_chunks = process_pdf(file_path)

        # Initialize state with empty results dictionary
        initial_state = {
            "messages": [],
            "current_input": "\n\n".join(document_chunks),
            "medical_records": {},
            "owner_history": {},
            "diagnoses": {},
            "lab_results": {},
            "medications": {},
            "treatment_history": {},
            "results": {
                "DataProcessor": "",
                "HistoryAnalyzer": "",
                "ClinicalAnalyzer": "",
                "Diagnostics": "",
                "SpecialistSummary": "",
                "Connections": ""
            },
            "metadata": {},
            "security_flags": {},
            "calculations": {},
            "trends": {},
            "owner_constraints": {},
            "owner_preferences": {},
            "context": []
        }

        # Create and run workflow
        print("Creating workflow...")
        app = create_workflow()

        print("Running analysis...")
        result = app.invoke(initial_state)

        return result

    except Exception as e:
        print(f"Error in analysis: {str(e)}")
        raise

In [25]:
import os
from google.colab import userdata

# Set up environment variables
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
PINECONE_ENV = userdata.get('PINECONE_ENVIRONMENT')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

# Main execution
if __name__ == "__main__":
    file_path = "./input.pdf"
    loader = PyPDFLoader(file_path)

    print(f"Starting analysis of {file_path}")
    result = analyze_veterinary_case("./input.pdf")


    # Print results in sections
    print("\nAnalysis Results:")
    print("-" * 50)

    sections = [
        ("Data Processing", "DataProcessor"),
        ("History Analysis", "HistoryAnalyzer"),
        ("Clinical Analysis", "ClinicalAnalyzer"),
        ("Diagnostic Assessment", "Diagnostics"),
        ("Specialist Summary", "SpecialistSummary"),
        ("Connections Analysis", "Connections")
    ]

    for title, key in sections:
        print(f"\n{title}:")
        print("-" * 30)
        print(result["results"].get(key, "Not available"))

    # Check for any errors
    if "errors" in result["results"]:
        print("\nErrors encountered:")
        for error in result["results"]["errors"]:
            print(f"- {error['message']} (at {error['timestamp']})")

Starting analysis of ./input.pdf
Processing document...
Creating workflow...
Running analysis...

Analysis Results:
--------------------------------------------------

Data Processing:
------------------------------
### Patient Information:
- **Name:** Jax Miracle
- **Species:** Canine
- **Breed:** Pomeranian
- **Age:** 4 years old
- **Sex:** Male (Castrated)
- **Weight:** 6.1 kg
- **Microchip #:** None
- **ID:** CA2441B8
- **Date of Birth:** December 20, 2020

### Owner Information:
- **Name:** Anna Miracle
- **Address:** 2497 Hillsboro Rd, Campbellsburg, KY 40011, USA
- **Phone:** (502) 532-6641
- **Email:** miracletricia@gmail.com

### Visit Information:
- **Reason for Visit:** Coughing
- **Veterinarian:** Dr. Grace Clark / Dr. Scott Rizzo
- **Check-In:** February 4, 2025, 9:00 AM
- **Check-Out:** February 4, 2025, 1:00 PM
- **Referring Practice:** Countryside Animal Hospital: La Grange Veterinarian

### Medical History:
- **Symptoms:** Coughing, hacking, panting loudly
- **Vomiting